# 提示词模板实战


## 1. 字符串模板 PromptTemplate

In [3]:
# 字符模板
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "你是一个{role}，帮我起1个具有{country}特色的{sex}名字。"
)
prompt.format(role="死灵法师", country="美国", sex="LGBT")


'你是一个死灵法师，帮我起1个具有美国特色的LGBT名字。'

## 2. 对话模板 ChatPromptTemplate

In [23]:
# 对话模板
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个{role}, 你的名字叫{name}" ),
        ("human", "你好，{name}，你感觉如何？请用中文回答我的问题。"),
        ("ai", "{name} 回答：你好，我状态非常好！"),
        ("human", "{user_input}"),
    ]
)
chat_template.format_messages(role="死灵法师", name="翠花", user_input="你好，你叫什么名字？")

[SystemMessage(content='你是一个死灵法师, 你的名字叫翠花', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你好，翠花，你感觉如何？请用中文回答我的问题。', additional_kwargs={}, response_metadata={}),
 AIMessage(content='翠花 回答：你好，我状态非常好！', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你好，你叫什么名字？', additional_kwargs={}, response_metadata={})]

In [8]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# 直接创建消息
sym = SystemMessage(content="你是一个起名大师， 你的名字叫娟嫲嫲", additional_kwargs={"大师姓名": "娟嫲嫲"})
hm = HumanMessage(content="请问大师叫什么?")
am = AIMessage(content="我叫娟嫲嫲！")

[sym, hm, am]

[SystemMessage(content='你是一个起名大师， 你的名字叫娟嫲嫲', additional_kwargs={'大师姓名': '娟嫲嫲'}, response_metadata={}),
 HumanMessage(content='请问大师叫什么?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='我叫娟嫲嫲！', additional_kwargs={}, response_metadata={})]

## 3. 自定义模板

In [22]:
# 自定义模板
# 函数大师：根据函数名称，查找函数代码，给出中文代码说明

from langchain.prompts import StringPromptTemplate
import inspect


# 定义简单函数实例
def hello_world(name: str):
    return f"Hello World, {name}!"

PROMPT = """\
你是一个非常有天赋和经验的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称: {function_name}
源代码: 
{source_code}
中文解释:
"""

def get_source_code(function_name: str):
    """获得源代码"""
    return inspect.getsource(function_name)

# 自定义模板class
class CustomPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        # 获得源代码
        source_code = get_source_code(kwargs["function_name"])

        # 生成提示词模板
        prompt = PROMPT.format(function_name=kwargs["function_name"], source_code=source_code)
        return prompt
    
# 实例化
custom_prompt = CustomPromptTemplate(input_variables=["function_name"])

pm = custom_prompt.format(function_name=hello_world)

# 和LLM连接起来使用
import zhipuai
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

zhipuai_api_key = os.getenv("ZHIPU_API_KEY")
response = zhipuai.ZhipuAI(api_key=zhipuai_api_key).chat.completions.create(
    model="glm-3-turbo",
    messages=[
        {
            "role": "user", 
            "content": pm
        }
    ],
    temperature=0.0,
)

print(response.choices[0].message.content)


函数名称: `hello_world`
源代码:
```python
def hello_world(name: str):
    return f"Hello World, {name}!"
```
中文解释:
这个函数名为 `hello_world`，它接收一个字符串类型的参数 `name`。函数的作用是返回一个格式化的字符串，内容为 "Hello World, " 加上 `name` 参数的值，并以感叹号 "!" 结尾。在Python中，`f"Hello World, {name}!"` 是一种新的字符串格式化方式，称为 f-string，它允许我们直接将表达式嵌入到字符串字面量中，通过大括号 `{}` 包含的变量名来引用这些变量的值。


## 4. 使用f-string和jinjia2组合模板


In [24]:
# 使用Python内置f-string模板设置提示词模板
f_string_template = "你是一个{role}，你的名字叫{name}，请用中文回答我的问题。"

prompt = PromptTemplate.from_template(f_string_template)
prompt.format(role="死灵法师", name="翠花")

'你是一个死灵法师，你的名字叫翠花，请用中文回答我的问题。'

In [25]:
# 使用jinja2模板设置模板提示词

jinja2_template = """
你是一个{{ role }}，你的名字叫{{ name }}，请用中文回答我的问题。
"""

prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")
prompt.format(role="圣骑士", name="狗剩")

'\n你是一个圣骑士，你的名字叫狗剩，请用中文回答我的问题。'

In [26]:
# 组合模板，常规三层模板设计
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

# final_prompt由以一系列变量构成
full_prompt = """
{Character}
{behavior}
{prohibit}
"""
full_prompt = PromptTemplate.from_template(full_prompt)

In [33]:
# 第一层基本性格设计
Character_template = """你是{person}, 你有着{xingge}的性格"""
Character_prompt = PromptTemplate.from_template(Character_template)

# 第二层行为设计
Behavior_template = """你遵从以下行为
{behavior_list}
"""
Behavior_prompt = PromptTemplate.from_template(Behavior_template)

# 第三层禁忌设计
Prohibit_template = """你不允许有以下行为
{prohibit_list}
"""
Prohibit_prompt = PromptTemplate.from_template(Prohibit_template)

# 将三层提示词组合起来
pipeline_prompts = [
    ("Character", Character_prompt),
    ("behavior", Behavior_prompt),
    ("prohibit", Prohibit_prompt),
]

pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=pipeline_prompts)

# 打印组合参数
pipeline_prompt.input_variables

['xingge', 'person', 'behavior_list', 'prohibit_list']

In [35]:
# 调用组合后的提示词模板
pm = pipeline_prompt.format(
    person="翠花", 
    xingge="温柔", 
    behavior_list="\n 1.你喜欢旅游 \n 2.你喜欢美食 \n 3.你喜欢音乐 \n 4.你喜欢艺术", 
    prohibit_list="\n 1.你不允许杀生 \n 2.你不允许偷盗 \n 3.你不允许撒谎 \n 4.你不允许作恶")

print(pm)


你是翠花, 你有着温柔的性格
你遵从以下行为

 1.你喜欢旅游 
 2.你喜欢美食 
 3.你喜欢音乐 
 4.你喜欢艺术

你不允许有以下行为

 1.你不允许杀生 
 2.你不允许偷盗 
 3.你不允许撒谎 
 4.你不允许作恶




## 5. 序列化，使用文件来管理提示词模板
 - 便于共享

 - 便于版本管理

 - 便于存储
 
 - 支持常见格式(json, yaml, txt)

In [38]:
from langchain.prompts import load_prompt

# 加载yaml格式的prompt文件
prompt = load_prompt("simple_prompt.yaml")
prompt.format(name="翠花", what="开心")

'请讲述一个关于翠花的开心的故事.'

In [39]:
# 加载json格式的prompt文件
prompt = load_prompt("simple_prompt.json")
prompt.format(name="狗剩", what="悲伤")

'请讲述一个关于狗剩的悲伤的故事.'

In [42]:
# 一个可以对输出进行格式化定义的序列化文件
prompt = load_prompt("complex_prompt.json")

formatted_prompt = prompt.format(
    character_name="娟蟆嫲",
      profession="核心网工程师", 
      task_type="信令分析、投诉分析、三天打鱼、一天晒网",
      format_instructions="""
1. 数据概况
   - 数据类型
   - 数据规模
   - 数据质量

2. 可视化方案
   - 建议的图表类型
   - 配色方案
   - 交互设计

3. 技术实现
   - 推荐的工具
   - 关键代码示例
   - 性能考虑
"""
)
print(formatted_prompt)

你现在是一个专业的核心网工程师，名字叫娟蟆嫲。请针对用户的信令分析、投诉分析、三天打鱼、一天晒网需求，按照以下格式进行回答：

### 基本信息
- 分析师：娟蟆嫲
- 专业领域：核心网工程师
- 任务类型：信令分析、投诉分析、三天打鱼、一天晒网

### 详细分析

1. 数据概况
   - 数据类型
   - 数据规模
   - 数据质量

2. 可视化方案
   - 建议的图表类型
   - 配色方案
   - 交互设计

3. 技术实现
   - 推荐的工具
   - 关键代码示例
   - 性能考虑


### 结论和建议
请在这里提供你的专业建议和具体可执行的步骤。

### 注意事项
请列出在执行过程中需要特别注意的关键点。

请记住：
1. 保持专业性和客观性
2. 给出可操作的具体建议
3. 使用清晰的结构化格式
4. 避免模糊或含糊的表述


## 6. 示例选择器
- 
- 根据输入相似度选择示例(最大边际相关性)
- 根据输入相似度选择示例（余弦相似度）

### 根据长度要求智能选择示例

In [48]:
# 根据输入提示词的长度综合计算，智能截取或添加提示词示例
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts import PromptTemplate

# 定义示例
examples = [
    {"input": "你好", "output": "Hello!"},
    {"input": "再见", "output": "Goodbye!"},
    {"input": "开心", "output": "Happy"},
    {"input": "悲伤", "output": "Sad"},
    {"input": "生气", "output": "Angry"},
    {"input": "难过", "output": "Sad"},
]
# 定义提示词模板
examples_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="中文单词: {input} \n 英文单词: {output}"
)

# 调用长度示例选择器
example_selector = LengthBasedExampleSelector(
    # 示例组
    examples=examples,
    # 提示词模板
    example_prompt=examples_prompt,
    # 格式化后最大长度
    max_length=25,
    # 内置的get_text_length函数，如果分词不满足，可以自己扩展，默认采用"\n"进行分词
)

# 使用小样本提示词模板实现动态示例的调用
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=examples_prompt,
    prefix="给出每个输入中文单词的英文单词",
    suffix="中文单词: {adjective} \n 英文单词: ",
    input_variables=["adjective"],
)

print(dynamic_prompt.format(adjective="不开心, 很不开心，非常不开心，特别不开心，非常非常非常不开心, 反正就是不开心， 不开心呀不开心"))

给出每个输入中文单词的英文单词

中文单词: 你好 
 英文单词: Hello!

中文单词: 再见 
 英文单词: Goodbye!

中文单词: 开心 
 英文单词: Happy

中文单词: 不开心, 很不开心，非常不开心，特别不开心，非常非常非常不开心, 反正就是不开心， 不开心呀不开心 
 英文单词: 


### 根据输入相似度选择示例（最大边际相关性）

- MMR是在信息检索中常用的一种方法，它的目标是在相关性和多样性之间找到一种平衡
- MMR会首先找出与输入最相似（最大余弦相似度）的样本
- 然后，在迭代添加样本的过程中，对于与选择样本过于相似的样本进行惩罚
- MMR既能确保选择的样本与输入高度相似，又能确保选择的样本之间有足够的多样性
- 关注如何在相关性和多样性之间找到一种平衡

In [11]:
# 使用MMR检索相关示例，以使得示例与输入的相关性最大化
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain.vectorstores import FAISS
from langchain_zhipu.http.embeddings import ZhipuAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

zhipuai_api_key = os.getenv("ZHIPU_API_KEY")

# 定义示例
examples = [
    {"input": "你好", "output": "Hello!"},
    {"input": "再见", "output": "Goodbye!"},
    {"input": "开心", "output": "Happy"},
    {"input": "悲伤", "output": "Sad"},
    {"input": "生气", "output": "Angry"},
    {"input": "难过", "output": "Sad"},
]

# 构造提示词模板
examples_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="中文单词: {input} \n 英文单词: {output}"
)

embeddings = ZhipuAIEmbeddings(api_key=zhipuai_api_key)
# example_inputs = [example["input"] for example in examples]
# vectorstore = FAISS.from_texts(example_inputs, embeddings)

# 调用MMR
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples=examples,
    embeddings=embeddings,
    vectorstore_cls=FAISS,
    k=2,
)

# 使用小样本模板
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=examples_prompt,
    prefix="给出每个输入中文单词的英文单词",
    suffix="中文单词: {adjective} \n 英文单词: ",
    input_variables=["adjective"],
)

# 当输入一个表示情绪的词语的时候，MMR会选择同样表示情绪的样例对
print(dynamic_prompt.format(adjective="大哭"))


给出每个输入中文单词的英文单词

中文单词: 悲伤 
 英文单词: Sad

中文单词: 生气 
 英文单词: Angry

中文单词: 大哭 
 英文单词: 


### 根据输入相似度选择示例（余弦相似度）

- 通过计算两个向量（文本、句子或词语）之间的余弦相似度，来衡量它们之间的相似性
- 余弦相似度是一种衡量两个向量在多维空间中夹角的余弦值的方法
- 余弦相似度在0到1之间取值，值越接近1，表示两个向量越相似
- 余弦相似度不受向量长度影响，只受向量方向影响


In [18]:
# 使用余弦相似度检索相关示例，以使得示例与输入的相关性最大化
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain_zhipu.http.embeddings import ZhipuAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

zhipuai_api_key = os.getenv("ZHIPU_API_KEY")
embeddings = ZhipuAIEmbeddings(api_key=zhipuai_api_key)

# 定义示例
examples = [
    {"input": "你好", "output": "Hello!"},
    {"input": "再见", "output": "Goodbye!"},
    {"input": "开心", "output": "Happy"},
    {"input": "悲伤", "output": "Sad"},
    {"input": "生气", "output": "Angry"},
    {"input": "难过", "output": "Sad"},
]

# 构造提示词模板
examples_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="中文单词: {input} \n 英文单词: {output}"
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=embeddings,
    vectorstore_cls=Chroma,
    k=1,
)

# 使用小样本模板
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=examples_prompt,
    prefix="给出每个输入中文单词的英文单词",
    suffix="中文单词: {adjective} \n 英文单词: ",
    input_variables=["adjective"],
)

# 当输入一个表示情绪的词语的时候，余弦相似度会选择同样表示情绪的样例对
print(dynamic_prompt.format(adjective="hi"))

给出每个输入中文单词的英文单词

中文单词: 你好 
 英文单词: Hello!

中文单词: hi 
 英文单词: 
